# Running Qwen3-next models with vLLM

This notebook provides a step-by-step guide on how to download and run `Qwen3-next` model using vLLM on NVIDIA GPUs for high-performance inference. vLLM is an open-source library that makes Large Language Model (LLM) inference and serving faster and more efficient by using an advanced memory management and continuous batching. It significantly increases model throughput, reduces GPU memory usage, and lowers infrastructure costs, making it a key tool for deploying LLMs at scale.

`Qwen3-next`is a brand-new mdoel architecture that introduces several key improvements over its predesessor: a hybrid attention mechanism, a highly sparse Mixture-of-Experts (MoE) structure, training-stability-friendly optimizations, and a multi-token prediction mechanism for faster inference. It is an 80-billion-parameter model that activates only 3 billion parameters during inference. Refer to the [model card](https://huggingface.co/Qwen/Qwen3-Next-80B-A3B-Instruct) for more details. The `Qwen3-next` has two variants:

- `Qwen3-Next-80B-A3B-Instruct`
- `Qwen3-Next-80B-A3B-Thinking`

#### Launch on NVIDIA Brev
You can simplify the environment setup by using [NVIDIA Brev](https://developer.nvidia.com/brev). Click the button below to launch this project on a Brev instance with the necessary dependencies pre-configured.

Once deployed, click on the "Open Notebook" button to get start with this guide

[![Launch on Brev](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-32vo0LNrj2pGPwXT7m5KYmqmJ0y)

## Prerequisites

### Hardware
To run the `Qwen3-Next-80B-A3B-Instruct` model, you will need an 4x A100 or 4xH200 NVIDIA 

### Software
- CUDA Toolkit 12.8 or later
- Python 3.12 or later
- vllm 0.10.x

## Installing vLLM

To run `Qwen3-Next` models you will need to install the nightly build of vLLM. 

### Python checking

Checking python version and paths

In [ ]:
import sys
print(sys.version)
print(sys.executable)

In [ ]:
!python -m pip install -U pip wheel setuptools uv

In [ ]:
# Install vLLM via pip
%uv pip install vllm --torch-backend=auto --extra-index-url https://wheels.vllm.ai/nightly

In [ ]:
# GPU environment check
import torch
import platform

print(f"Python: {platform.python_version()}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Num GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"GPU[{i}]: {props.name} | SM count: {props.multi_processor_count} | Mem: {props.total_memory / 1e9:.2f} GB")

## Launch Model with Multi-GPU setup

There are multiple ways to load the model. 
- Activating a server using vllm serve
- vLLM python client

For `vllm serve` You can run the below command from command line or use subprocess to run it from cell. 
`vllm serve Qwen/Qwen3-Next-80B-A3B-Instruct \`
  `--tensor-parallel-size 4 \`
  `--served-model-name qwen3-next `

We will running it from the cell and hence will be wrapping it with subprocess

### Wait around 5 minutes

In [ ]:
import subprocess

cmd = [
    "vllm", "serve", "Qwen/Qwen3-Next-80B-A3B-Instruct",
    "--tensor-parallel-size", "4",
    "--served-model-name", "qwen3-next",
    "--host", "0.0.0.0", "--port", "8000"
]

subprocess.Popen(cmd)

## Inferencing using vllm server

### Inference using vllm server

In [ ]:
import requests

def inference(user_prompt):
    url = "http://localhost:8000/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "qwen3-next",
        "messages": [
            {"role": "user", "content": user_prompt}
        ]
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [ ]:
# Usage example

user_prompt = " What is the capital of France and why do people travel go there? "
output = inference(user_prompt)
result = output['choices'][0]['message']['content']
print(result)

### Inference using vLLM python client

## Closing the previous instance (we have to look better way than kill process)

In [ ]:
!ps aux | grep vllm
!kill -9 41393

In [ ]:
import os
from vllm import LLM, SamplingParams

MODEL_ID = "Qwen/Qwen3-Next-80B-A3B-Instruct"

llm = LLM(
    model=MODEL_ID,
    dtype="bfloat16",
    trust_remote_code=True,
    max_model_len=65536,
    gpu_memory_utilization=0.95,
    tensor_parallel_size=4,
)

print("Model ready")

### Generate: single and batch

In [ ]:
params = SamplingParams(temperature=0.6, max_tokens=200)

# Single prompt
single = llm.generate(["What is Nemotron Super?"], sampling_params=params)
print(single[0].outputs[0].text)

# Batch prompts
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "Explain quantum computing in simple terms:"
]
outputs = llm.generate(prompts, sampling_params=params)
for i, out in enumerate(outputs):
    print(f"\nPrompt {i+1}: {out.prompt!r}")
    print(out.outputs[0].text)

# Conclusion and Next Steps
Congratulations! You successfully deployed `Qwen3-Next` using vLLM.

In this notebook, you have learned how to:
- Set up your environment with the necessary dependencies.
- Use vllm serve to deploy the model.
- Run inference.